In [8]:
!pip install jupyter-dash

In [11]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode='inline', port = 7777)

---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
~/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/flask/app.py in wsgi_app(
    self=<Flask '__main__'>,
    environ={'HTTP_ACCEPT': '*/*', 'HTTP_ACCEPT_ENCODING': 'gzip, deflate, br', 'HTTP_CONNECTION': 'keep-alive', 'HTTP_HOST': '127.0.0.1:7777', 'HTTP_USER_AGENT': 'python-requests/2.26.0', 'PATH_INFO': '/_alive_d78a3cda-5f10-4fb4-8976-c301129b6b96', 'QUERY_STRING': '', 'RAW_URI': '/_alive_d78a3cda-5f10-4fb4-8976-c301129b6b96', 'REMOTE_ADDR': '127.0.0.1', 'REMOTE_PORT': 58369, ...},
    start_response=<function WSGIRequestHandler.run_wsgi.<locals>.start_response>
)
   2071             try:
   2072                 ctx.push()
-> 2073                 response = self.full_dispatch_request()
        response = undefined
        self.full_dispatch_request = <bound method Flask.full_dispatch_request of <Flask '__main__'>>
   2074

OSError: Address 'http://127.0.0.1:7777' already in use.
    Try passing a different port to run_server.